# Where is your GAN? Генеративно-состязательная нейросеть: ваша первая модель на PyTorch

Данный материал представляет собой незначительно сокращенный перевод публикации Ренато Кандидо [Generative Adversarial Networks: Build Your First Models](https://realpython.com/generative-adversarial-networks/).

Генеративно-состязательные сети (англ. Generative adversarial networks, сокр. GAN) – [нейронные сети](https://ru.wikipedia.org/wiki/%D0%9D%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C), которые умеют генерировать изображения, музыку, речь и тексты, похожие на те, что делают люди. 

GAN были активной темой исследований последних лет. Директор по исследованиям в области искусственного интеллекта в Facebook Ян Лекан назвал состязательное обучение «самой интересной идеей в области машинного обучения за последние 10 лет». Ниже вы узнаете, как работают GAN и создадите две собственные модели. Для работы с моделями будет использоваться [фреймворк глубокого обучения](https://proglib.io/p/dl-frameworks) PyTorch.

# Что такое генеративно-состязательная нейросеть?

[Генеративно-состязательные сети](https://ru.wikipedia.org/wiki/%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%BE-%D1%81%D0%BE%D1%81%D1%82%D1%8F%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C) – это модели машинного обучения, умеющие имитировать заданное распределение данных. Впервые они были предложены в [статье NeurIPS](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf) 2014 г. экспертом в глубоком обучении Яном Гудфеллоу и его коллегами.

GAN состоят из двух нейронных сетей, одна из которых обучена генерировать данные, а другая обучена отличать ложные данные от реальных (отсюда и «состязательный» характер модели). GAN показывают впечатляющие результаты в отношении генерации изображений и видео, такие как:
* Перенос стилей ([CycleGAN](https://github.com/junyanz/CycleGAN/)) – преобразование одного изображения в соответствии со стилем других изображений (например, картин известного художника)
* Генерация человеческих лиц ([StyleGAN](https://en.wikipedia.org/wiki/StyleGAN)), реалистичные примеры которых вы можете найти на сайте [This Person Does Not Exist](https://www.thispersondoesnotexist.com/).

GAN и другие структуры, генерирующие данные, называют **генеративными моделями** в противовес более широко изученным **дискриминативным моделям**. Прежде чем погрузиться в GAN, посмотрим на различия между этими двумя типами моделей.

# Сравнение дискриминативных и генеративных моделей

Дискриминативные модели используются для большинства [обучения с учителем](https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D1%81_%D1%83%D1%87%D0%B8%D1%82%D0%B5%D0%BB%D0%B5%D0%BC) на [классификацию](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D0%B8) или [регрессию](https://ru.wikipedia.org/wiki/%D0%A0%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7). В качестве примера проблемы классификации предположим, что вы хотите обучить [модель классификации изображений рукописных цифр от 0 до 9](https://proglib.io/p/neural-network-course). Для этого вы можете использовать маркированный набор данных, содержащий изображения рукописных цифр и связанные метки, указывающие соответствие цифр и изображений.

В процессе обучения для настройки параметров модели вы будете использовать специальный алгоритм. Его цель состоит в том, чтобы [минимизировать функцию потерь](https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F_%D0%BF%D0%BE%D1%82%D0%B5%D1%80%D1%8C) – критерий раскхождения между истинным значением оцениваемого параметра и его оценкой. После фазы обучения вы можете использовать модель для классификации нового изображения рукописной цифры, сопоставив входному изображению наиболее вероятную цифру, как показано на рисунке ниже.

![](https://files.realpython.com/media/fig_discriminative.9c22a1cd877d.png)
*Схема обучения дискриминативной модели*

Дискриминативную модель для задач классификации можно представить, как «черный ящик», который использует обучающие данные для изучения границ между классами. Найденные границы далее используются моделью, чтобы различить входные данные – предсказать их класс. В математическом отношении дискриминативные модели изучают [условную вероятность](https://ru.wikipedia.org/wiki/%D0%A3%D1%81%D0%BB%D0%BE%D0%B2%D0%BD%D0%B0%D1%8F_%D0%B2%D0%B5%D1%80%D0%BE%D1%8F%D1%82%D0%BD%D0%BE%D1%81%D1%82%D1%8C) $P(y|x)$ наблюдения $y$ при заданном входе $x$.

Дискриминативные модели это не обязательно нейронные сети. К ним также относятся такие модели машинного обучения, как [логистическая регрессия](https://ru.wikipedia.org/wiki/%D0%9B%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F) и [метод опорных векторов (SVM)](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%BE%D0%BF%D0%BE%D1%80%D0%BD%D1%8B%D1%85_%D0%B2%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D0%BE%D0%B2).

В то время как дискриминативные модели используются для контролируемого обучения, генеративные модели часто используют неразмеченный набор данных, то есть могут рассматриваться как форма [обучения без учителя](https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B1%D0%B5%D0%B7_%D1%83%D1%87%D0%B8%D1%82%D0%B5%D0%BB%D1%8F). Используя набор данных из рукописных цифр, вы можете обучить генеративную модель для генерации новых цифр. На этапе обучения модель использует определенный алгоритм для настройки параметров модели, чтобы также минимизировать функцию потерь и определить распределение вероятностей обучающего набора.

![](https://files.realpython.com/media/fig_generative.5f01c08f5208.png)
*Схема обучения генеративной модели*

В отличие от дискриминативных моделей, генеративные модели изучают свойства [функции вероятности](https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F_%D0%B2%D0%B5%D1%80%D0%BE%D1%8F%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B8) $P(x)$ входных данных $x$. В результате они порождают не предсказание, а новый объект со свойствами, родственными обучающему набору данных.

---

**Примечание**. Генеративные модели также можно использовать и для размеченных наборов данных. Их также можно использовать для задач классификации, но в целом дискриминативные модели работают лучше, когда речь идет о классификации. 

Вы можете найти больше информации об относительных сильных и слабых сторонах  классификаторов в статье [«Дискриминативные и генеративные классификаторы: сравнение логистической регрессии и наивного байесовского алгоритма»](https://realpython.com/generative-adversarial-networks/) (англ.).

---

Помимо GAN существуют другие генеративные модели архитектуры:
* [Машина Больцмана](https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%88%D0%B8%D0%BD%D0%B0_%D0%91%D0%BE%D0%BB%D1%8C%D1%86%D0%BC%D0%B0%D0%BD%D0%B0)
* [Автокодировщик](https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D1%82%D0%BE%D0%BA%D0%BE%D0%B4%D0%B8%D1%80%D0%BE%D0%B2%D1%89%D0%B8%D0%BA)
* [Скрытая марковская модель](https://ru.wikipedia.org/wiki/%D0%A1%D0%BA%D1%80%D1%8B%D1%82%D0%B0%D1%8F_%D0%BC%D0%B0%D1%80%D0%BA%D0%BE%D0%B2%D1%81%D0%BA%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C)
* Модели, предсказывающие следующее слово в последовательности, например, [GPT-2](https://en.wikipedia.org/wiki/OpenAI#GPT-2)

Тем не менее, в последнее время GAN привлекли наибольшее внимание общественности благодаря впечатляющим результатам в генерации изображений и видео. Поэтому остановимся на ее устройстве подробнее.

# Архитектура генеративно-состязательных нейросетей

Генеративно-состязательная сеть это на самом деле не одна сеть, а две: генератор и дискриминатор. Роль **генератора** состоит в том, чтобы на основе реальной выборки сгенерировать набор данных, напоминающий реальные данные. **Дискриминатор** в свою очередь обучен оценивать вероятность того, что данный образец получен из реальных данных, а не предоставлен генератором. Состязательность GAN заключается в том, что генератор и дискриминатор играют в кошки-мышки: генератор пытается обмануть дискриминатор, а дискриминатор старается лучше идентифицировать сгенерированные выборки.

Чтобы понять, как работает обучение GAN, рассмотрим игрушечный пример с набором данных, состоящим из двумерных выборок $(x_1, x_2)$, с $x_1$ в интервале от $0$ до $2π$ и $x_2 = sin(x_1)$, как показано на следующем рисунке.

![](https://files.realpython.com/media/fig_x1x2.f8a39d8ff58a.png)

Общая структура GAN для генерации пар $(x̃_1, x̃_2)$, напоминающих точки набора данных, показана на следующем рисунке.

![](https://files.realpython.com/media/fig_gan.4f0f744c7999.png)

Генератор $G$ получает на вход пары случайных чисел ($z_1, z_2$), преобразуя их так, чтобы они напоминали реальные выборки. Структура нейронной сети $G$ может быть произвольной, например, [многослойный персептрон](https://ru.wikipedia.org/wiki/%D0%9C%D0%BD%D0%BE%D0%B3%D0%BE%D1%81%D0%BB%D0%BE%D0%B9%D0%BD%D1%8B%D0%B9_%D0%BF%D0%B5%D1%80%D1%86%D0%B5%D0%BF%D1%82%D1%80%D0%BE%D0%BD_%D0%A0%D1%83%D0%BC%D0%B5%D0%BB%D1%8C%D1%85%D0%B0%D1%80%D1%82%D0%B0) (MLP) или [сверточная нейронная сеть](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C) (CNN).

На вход дискриминатора $D$ попеременно поступают реальные образцы из обучающего набора данных и смоделированные образцы, предоставленные генератором $G$. Роль дискриминатора заключается в оценке вероятности того, что входные данные принадлежат реальному набору данных. То есть обучение выполняется таким образом, чтобы $D$ выдавал $1$, когда получает реальный образец, и $0$, когда получает сгенерированный образец.

Как и в случае с генератором, вы можете выбрать любую структуру нейронной сети для $D$ с учетом размеров входных и выходных данных. В рассматриваемом примере ввод является двумерным, а выходные данные – [скаляром](https://ru.wikipedia.org/wiki/%D0%A1%D0%BA%D0%B0%D0%BB%D1%8F%D1%80%D0%BD%D0%B0%D1%8F_%D0%B2%D0%B5%D0%BB%D0%B8%D1%87%D0%B8%D0%BD%D0%B0) в диапазоне от 0 до 1.

Процесс обучения GAN заключается в [минимаксной игре](https://ru.wikipedia.org/wiki/%D0%9C%D0%B8%D0%BD%D0%B8%D0%BC%D0%B0%D0%BA%D1%81) двух игроков, в которой $D$ адаптирован для минимизации ошибки различия реального и сгенерированного образца, а $G$ адаптирован на максимизацию вероятности того, что $D$ допустит ошибку.

На каждом этапе обучения происходит обновление параметров моделей $D$ и $G$. Чтобы обучить $D$, на каждой итерации мы помечаем некоторую выбору реальных образцов из обучающих данных, как 1, а выборку сгенерированных образцов, созданных $G$, как 0. Таким образом, для обновления параметров $D$, как показано на следующей схеме, можно использовать обычную схему обучения с учителем.

![](https://files.realpython.com/media/fig_train_discriminator.cd1a1e32764f.png)
*Процесс обучения дискриминатора*

Для каждой партии обучающих данных, содержащих помеченные реальные и сгенерированные образцы, мы обновляем набор параметров модели $D$, минимизируя тем самым функцию потерь. После того, как параметры $D$ обновлены, мы обучаем $G$ генерировать более качественные образцы. Набор параметров D «замораживается» на время обучения генератора.

![](https://files.realpython.com/media/fig_train_generator.7196c4f382ba.png)

Когда $G$ будет генерировать образцы настолько хорошо, что $D$ начнет обманываться, выходная вероятность устремится к 1 – $D$ будет считать, что все образцы принадлежат к оригинальной выборке.

Теперь, когда вы знаете, как в работает GAN, мы готовы реализовать свой собственный вариант нейросети, используя популярный фреймворк глубокого обучения **PyTorch**.

# Ваша первая генеративно-состязательная нейросеть

В качестве первого эксперимента с порождающими состязательными сетями мы реализуем пример с синусоидой, описанный в предыдущем разделе.

Для запуска примера мы будем использовать библиотеку PyTorch, которую вы можете установить с помощью следующей [инструкции по установке](https://pytorch.org/get-started/locally/). Если вы уже серьезно заинтересовались Data Science, возможно вы уже использовали дистрибутив [Anaconda](https://www.anaconda.com/products/individual) и систему управления пакетами и средами [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html).

Для начала создайте окружение `conda` и активируйте его:

```Bash
$ conda create --name gan
$ conda activate gan
```

Здесь создается окружение `conda` с именем `gan`. Внутри созданной среды можно установить необходимые пакеты:


```Bash
$ conda install -c pytorch pytorch=1.4.0
$ conda install matplotlib jupyter
```

Поскольку PyTorch является очень активно разработанной средой, API в новых версиях может измениться. Примеры кода проверены для версии 1.4.0.

Помимо PyTorch, мы будем использовать Matplotlib для работы с графиками и [Jupyter Notebook](https://proglib.io/p/jupyter) для запуска кода в интерактивной среде. Это не обязательно, но облегчает работу над проектами машинного обучения.

Если вы будете использовать Jupyter Notebook, нужно предварительно зарегистрировать окружение `conda gan`, чтобы мы могли создавать блокноты, используя ее в качестве кернела. Для этого в активированной среде `gan` выполните следующую команду:

```Bash
$ python -m ipykernel install --user --name gan
```

Начнём с импорта необходимых библиотек:

In [8]:
import torch
from torch import nn

import math
import matplotlib.pyplot as plt

Здесь мы импортируем библиотеку PyTorch как `torch`. Из библиотеки мы отдельно импортируем компонент `nn` просто чтобы обращения к параметрам были более компактными. Затем мы импортируем `math` для получения значения константы `pi` и инструмент построения графиков Matplotlib.

Хорошей практикой является установка генератора случайных чисел так, чтобы эксперимент можно было одинаково воспроизвести на любой машине. Чтобы сделать это в PyTorch, запустите следующий код:

In [ ]:
torch.manual_seed(111)

Число 111 представляет случайное начальное число, используемое для инициализации генератора случайных чисел, который используется для инициализации весов нейронной сети. Несмотря на случайный характер эксперимента, он должен давать те же результаты, пока используется одно и то же семя.

Теперь, когда среда настроена, вы можете подготовить данные обучения.

# Подготовка данных для обучения

Обучающие данные состоят из пар (x₁, x₂), так что x₂ состоит из значения синуса x₁ для x₁ в интервале от 0 до 2π. Вы можете реализовать это следующим образом:

In [ ]:
train_data_length = 1024
train_data = torch.zeros((train_data_length, 2))
train_data[:, 0] = 2 * math.pi * torch.rand(train_data_length)
train_data[:, 1] = torch.sin(train_data[:, 0])
train_labels = torch.zeros(train_data_length)
train_set = [
    (train_data[i], train_labels[i]) for i in range(train_data_length)
]

Здесь вы составляете тренировочный набор с 1024 парами (x₁, x₂). В строке 2 вы инициализируете train_data, тензор с размерами 1024 строки и 2 столбца, все из которых содержат нули. Тензор - это многомерный массив, похожий на массив NumPy.

В строке 3 вы используете первый столбец train_data для хранения случайных значений в интервале от 0 до 2π. Затем в строке 4 вы вычисляете второй столбец тензора как синус первого столбца.

Далее вам понадобится тензор меток, которые требуются загрузчику данных PyTorch. Поскольку в GAN используются неконтролируемые методы обучения, метки могут быть любыми. Они не будут использованы, в конце концов.

В строке 5 вы создаете train_labels, тензор, заполненный нулями. Наконец, в строках с 6 по 8 вы создаете train_set в виде списка кортежей, где каждый ряд train_data и train_labels представлен в каждом кортеже, как и ожидалось загрузчиком данных PyTorch.

Вы можете изучить данные тренировки, нанеся на график каждую точку (x₁, x₂):

In [1]:
plt.plot(train_data[:, 0], train_data[:, 1], ".")

NameError: name 'plt' is not defined

Вывод должен быть похож на следующий рисунок:

![](https://files.realpython.com/media/fig_train_sin_mpl.2e194bac5580.png)

С помощью train_set вы можете создать загрузчик данных PyTorch:

In [ ]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)

Здесь вы создаете загрузчик данных с именем train_loader, который будет перетасовывать данные из train_set и возвращать пакеты из 32 сэмплов, которые вы будете использовать для обучения нейронных сетей.

После настройки обучающих данных вам необходимо создать нейронные сети для дискриминатора и генератора, которые будут составлять GAN. В следующем разделе вы реализуете дискриминатор.

# Реализация Дискриминатора
В PyTorch модели нейронной сети представлены классами, которые наследуются от nn.Module, поэтому вам придется определить класс для создания дискриминатора. Для получения дополнительной информации об определении классов взгляните на объектно-ориентированное программирование (ООП) в Python 3.

Дискриминатор - это модель с двумерным входом и одномерным выходом. Он получит выборку из реальных данных или от генератора и предоставит вероятность того, что выборка относится к реальным обучающим данным. Код ниже показывает, как создать дискриминатор:

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        output = self.model(x)
        return output

Вы используете .__ init __ () для построения модели. Во-первых, вам нужно вызвать super () .__ init __ () для запуска .__ init __ () из nn.Module. Используемый дискриминатор - это нейронная сеть MLP, определенная последовательно с использованием nn.Sequential (). Он имеет следующие характеристики:

Строки 5 и 6: вход двумерный, а первый скрытый слой состоит из 256 нейронов с активацией ReLU.

Строки 8, 9, 11 и 12: второй и третий скрытые слои состоят из 128 и 64 нейронов, соответственно, с активацией ReLU.

Строки 14 и 15: выход состоит из одного нейрона с сигмоидальной активацией для представления вероятности.

Строки 7, 10 и 13: после первого, второго и третьего скрытых слоев вы используете выпадение, чтобы избежать переобучения.

Наконец, вы используете .forward (), чтобы описать, как рассчитывается вывод модели. Здесь x представляет вход модели, который является двумерным тензором. В этой реализации выходные данные получаются путем подачи входных данных x к определенной вами модели без какой-либо другой обработки.

После объявления класса дискриминатора вы должны создать экземпляр объекта Discriminator:

In [ ]:
discriminator = Discriminator()

discriminator представляет собой пример нейронной сети, которую вы определили и готов к обучению. Однако, прежде чем вы реализуете обучающий цикл, вашей GAN также нужен генератор. Вы реализуете один в следующем разделе.

Реализация генератора
В порождающих состязательных сетях генератор - это модель, которая берет выборки из скрытого пространства в качестве входных данных и генерирует данные, напоминающие данные в обучающем наборе. В данном случае это модель с двумерным вводом, которая будет получать случайные точки (z₁, z₂), и двумерный вывод, который должен обеспечивать (x̃₁, x̃₂) точки, похожие на те, которые получены из обучающих данных.

Реализация похожа на то, что вы сделали для дискриминатора. Сначала вам нужно создать класс Generator, который наследуется от nn.Module, определяя архитектуру нейронной сети, а затем вам нужно создать экземпляр объекта Generator:

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )

    def forward(self, x):
        output = self.model(x)
        return output

generator = Generator()

Здесь генератор представляет собой генератор нейронной сети. Он состоит из двух скрытых слоев с 16 и 32 нейронами, оба с активацией ReLU, и линейного слоя активации с 2 нейронами на выходе. Таким образом, выходные данные будут состоять из вектора с двумя элементами, которые могут иметь любое значение в диапазоне от отрицательной бесконечности до бесконечности, которое будет представлять (x̃₁, x̃₂).

Теперь, когда вы определили модели для дискриминатора и генератора, вы готовы выполнить обучение!

Обучение моделей
Перед тренировкой моделей необходимо настроить некоторые параметры, которые будут использоваться во время тренировки:

In [ ]:
lr = 0.001
num_epochs = 300
loss_function = nn.BCELoss()

Здесь вы устанавливаете следующие параметры:

В строке 1 задается скорость обучения (lr), которую вы будете использовать для адаптации весов сети.

В строке 2 задается количество эпох (num_epochs), которое определяет, сколько повторений обучения будет выполнено с использованием всего обучающего набора.

В строке 3 переменная loss_function назначается двоичной кросс-энтропийной функции BCELoss (), которая является функцией потерь, которую вы будете использовать для обучения моделей.

Двоичная функция кросс-энтропии является подходящей функцией потерь для обучения дискриминатора, потому что она рассматривает задачу двоичной классификации. Он также подходит для обучения генератора, поскольку он подает свой вывод на дискриминатор, который обеспечивает двоичный наблюдаемый вывод.

PyTorch реализует различные правила обновления веса для обучения модели в torch.optim. Вы будете использовать алгоритм Адама для обучения моделей дискриминатора и генератора. Чтобы создать оптимизаторы с помощью torch.optim, запустите следующие строки:

In [ ]:
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

Наконец, вам необходимо реализовать обучающий цикл, в котором обучающие образцы подаются на модели, а их веса обновляются, чтобы минимизировать функцию потерь:

In [ ]:
for epoch in range(num_epochs):
    for n, (real_samples, _) in enumerate(train_loader):
        # Data for training the discriminator
        real_samples_labels = torch.ones((batch_size, 1))
        latent_space_samples = torch.randn((batch_size, 2))
        generated_samples = generator(latent_space_samples)
        generated_samples_labels = torch.zeros((batch_size, 1))
        all_samples = torch.cat((real_samples, generated_samples))
        all_samples_labels = torch.cat(
            (real_samples_labels, generated_samples_labels)
        )

        # Training the discriminator
        discriminator.zero_grad()
        output_discriminator = discriminator(all_samples)
        loss_discriminator = loss_function(
            output_discriminator, all_samples_labels)
        loss_discriminator.backward()
        optimizer_discriminator.step()

        # Data for training the generator
        latent_space_samples = torch.randn((batch_size, 2))

        # Training the generator
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)
        output_discriminator_generated = discriminator(generated_samples)
        loss_generator = loss_function(
            output_discriminator_generated, real_samples_labels
        )
        loss_generator.backward()
        optimizer_generator.step()

        # Show loss
        if epoch % 10 == 0 and n == batch_size - 1:
            print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
            print(f"Epoch: {epoch} Loss G.: {loss_generator}")

Для GAN вы обновляете параметры дискриминатора и генератора на каждой обучающей итерации. Как обычно делается для всех нейронных сетей, учебный процесс состоит из двух циклов, один для тренировочных эпох, а другой для партий для каждой эпохи. Внутри внутреннего цикла вы начинаете подготовку данных для обучения дискриминатора:

Строка 2: вы получаете реальные образцы текущей партии из загрузчика данных и назначаете их в real_samples. Обратите внимание, что первое измерение тензора имеет количество элементов, равное batch_size. Это стандартный способ организации данных в PyTorch, где каждая строка тензора представляет один образец из пакета.

Строка 4. Вы используете torch.ones () для создания меток со значением 1 для реальных образцов, а затем назначаете метки для real_samples_labels.

Строки 5 и 6: вы создаете сгенерированные сэмплы, сохраняя случайные данные в latent_space_samples, которые затем передаете в генератор для получения generate_samples.

Строка 7: вы используете torch.zeros (), чтобы присвоить значение 0 меткам для сгенерированных сэмплов, а затем сохраните метки в generate_samples_labels.

Строки с 8 по 11: вы объединяете реальные и сгенерированные образцы и метки и сохраняете их в all_samples и all_samples_labels, которые вы будете использовать для обучения дискриминатора.

Далее в строках с 14 по 19 вы тренируете дискриминатор:

Строка 14: в PyTorch необходимо очищать градиенты на каждом этапе обучения, чтобы избежать их накопления. Вы делаете это с помощью .zero_grad ().

Строка 15: Вы вычисляете выходные данные дискриминатора, используя обучающие данные в all_samples.

Строки 16 и 17: Вы вычисляете функцию потерь, используя выходные данные модели в output_discriminator и метки в all_samples_labels.

Строка 18: Вы вычисляете градиенты для обновления весов с помощью loss_discriminator.backward ().

Строка 19: Вы обновляете веса дискриминатора, вызывая optimizer_discriminator.step ().

Далее в строке 22 вы подготавливаете данные для обучения генератора. Вы храните случайные данные в latent_space_samples, с количеством строк, равным batch_size. Вы используете два столбца, поскольку вы предоставляете двумерные данные в качестве входных данных для генератора.

Вы тренируете генератор в строках с 25 по 32:

Строка 25: вы очищаете градиенты с помощью .zero_grad ().

Строка 26: вы передаете генератору latent_space_samples и сохраняете его выходные данные в generate_samples.

Строка 27: вы передаете выходные данные генератора в дискриминатор и сохраняете его выходные данные в output_discriminator_generated, который вы будете использовать в качестве выходных данных всей модели.

Строки с 28 по 30: Вы вычисляете функцию потерь, используя выходные данные системы классификации, сохраненные в output_discriminator_generated, и метки в real_samples_labels, которые все равны 1.

Строки 31 и 32: Вы рассчитываете градиенты и обновляете веса генератора. Помните, что когда вы обучали генератор, вы сохраняли вес дискриминатора замороженным, так как вы создали optimizer_generator с его первым аргументом, равным generator.parameters ().

Наконец, в строках с 35 по 37 отображаются значения функций потерь дискриминатора и генератора в конце каждых десяти эпох.

Поскольку модели, используемые в этом примере, имеют несколько параметров, обучение будет завершено через несколько минут. В следующем разделе вы будете использовать обученный GAN для генерации некоторых образцов.

# Проверка образцов, сгенерированных GAN

Генеративные состязательные сети предназначены для генерации данных. Таким образом, после того, как процесс обучения завершен, вы можете получить несколько случайных выборок из скрытого пространства и передать их в генератор для получения некоторых сгенерированных выборок:

In [ ]:
latent_space_samples = torch.randn(100, 2)
generated_samples = generator(latent_space_samples)

Затем вы можете построить сгенерированные образцы и проверить, не похожи ли они на тренировочные данные. Перед построением графика созданных данных сгенерированных образцов вам необходимо использовать .detach(), чтобы вернуть тензор из вычислительного графа PyTorch, который вы затем будете использовать для вычисления градиентов:

In [ ]:
generated_samples = generated_samples.detach()
plt.plot(generated_samples[:, 0], generated_samples[:, 1], ".")

Вы можете видеть, как распределение сгенерированных данных напоминает распределение реальных данных. Используя фиксированный тензор скрытых проб пространства и подавая его на генератор в конце каждой эпохи в процессе обучения, вы можете визуализировать развитие обучения:

![](https://files.realpython.com/media/fig_gan_x1x2.69b1d6021da8.gif)

Обратите внимание, что в начале процесса обучения распределение сгенерированных данных сильно отличается от реальных данных. Однако, по мере обучения, генератор изучает реальное распределение данных.

Теперь, когда вы выполнили свою первую реализацию генеративной состязательной сети, вы перейдете к более практичному применению с использованием изображений.

Генератор рукописных цифр с GAN
Генеративные состязательные сети могут также генерировать высокоразмерные образцы, такие как изображения. В этом примере вы собираетесь использовать GAN для создания изображений рукописных цифр. Для этого вы будете обучать модели, используя набор данных MNIST из рукописных цифр, который включен в пакет torchvision.

Для начала вам необходимо установить torchvision в активированной среде gan conda:

```bash
$ conda install -c pytorch torchvision=0.5.0
```

Опять же, вы используете конкретную версию torchvision, чтобы обеспечить выполнение примера кода, как вы это делали с pytorch. После настройки среды вы можете приступить к реализации моделей в Jupyter Notebook. Откройте его и создайте новую записную книжку, щелкнув New и выбрав gan.

Как и в предыдущем примере, вы начинаете с импорта необходимых библиотек:

In [2]:
import torch
from torch import nn

import math
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms

Помимо библиотек, которые вы импортировали ранее, вам понадобится torchvision и преобразования для получения обучающих данных и преобразования изображений.

Опять же, установите начальный генератор случайных чисел, чтобы иметь возможность повторить эксперимент:

In [3]:
torch.manual_seed(111)

Поскольку в этом примере используются изображения в обучающем наборе, модели должны быть более сложными с большим количеством параметров. Это замедляет процесс обучения, занимая около двух минут на эпоху при работе на CPU. Вам понадобится около пятидесяти эпох, чтобы получить соответствующий результат, поэтому общее время обучения при использовании процессора составляет около ста минут.

Чтобы сократить время обучения, вы можете использовать графический процессор для обучения модели, если она у вас есть. Однако вам придется вручную перемещать тензоры и модели в графический процессор, чтобы использовать их в процессе обучения.

Вы можете убедиться, что ваш код будет работать при любой установке, создав объект устройства, который указывает либо на ЦП, либо, если он доступен, на графический процессор:

In [ ]:
device = ""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

Позже вы будете использовать это устройство, чтобы указать, где следует создавать тензоры и модели, используя графический процессор, если таковой имеется.

Теперь, когда базовая среда установлена, вы можете подготовить данные обучения.

# Подготовка данных обучения
Набор данных MNIST состоит из изображений в градациях серого 28 × 28 пикселей, написанных от руки цифрами от 0 до 9. Чтобы использовать их с PyTorch, вам потребуется выполнить некоторые преобразования. Для этого вы определяете функцию transform, которая будет использоваться при загрузке данных:

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

Функция состоит из двух частей:

transforms.ToTensor () преобразует данные в тензор PyTorch.
transforms.Normalize () преобразует диапазон тензорных коэффициентов.
Исходные коэффициенты, заданные функцией transforms.ToTensor (), находятся в диапазоне от 0 до 1, и поскольку фоны изображения черные, большинство коэффициентов равны 0, когда они представлены с использованием этого диапазона.

transforms.Normalize () изменяет диапазон коэффициентов от -1 до 1, вычитая 0,5 из исходных коэффициентов и деля результат на 0,5. Благодаря этому преобразованию количество элементов, равное 0 во входных выборках, значительно сокращается, что помогает в обучении моделей.

Аргументы transforms.Normalize () - это два кортежа (M₁, ..., Mₙ) и (S₁, ..., Sₙ), где n представляет количество каналов изображений. Изображения в градациях серого, такие как в наборе данных MNIST, имеют только один канал, поэтому кортежи имеют только одно значение. Затем для каждого канала i изображения transforms.Normalize () вычитает Mᵢ из коэффициентов и делит результат на Sᵢ.

Теперь вы можете загрузить обучающие данные с помощью torchvision.datasets.MNIST и выполнить преобразования с помощью transform:

In [ ]:
train_set = torchvision.datasets.MNIST(
    root=".", train=True, download=True, transform=transform
)

Аргумент download = True гарантирует, что при первом запуске приведенного выше кода набор данных MNIST будет загружен и сохранен в текущем каталоге, как указано в аргументе root.

Теперь, когда вы создали train_set, вы можете создать загрузчик данных, как вы делали это раньше:

In [ ]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)

Вы можете использовать Matplotlib для построения некоторых образцов тренировочных данных. Чтобы улучшить визуализацию, вы можете использовать cmap = gray_r, чтобы изменить цветовую карту и нарисовать цифры черным цветом на белом фоне:

In [ ]:
real_samples, mnist_labels = next(iter(train_loader))
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(real_samples[i].reshape(28, 28), cmap="gray_r")
    plt.xticks([])
    plt.yticks([])

Вывод должен быть примерно таким:

![](https://files.realpython.com/media/fig_train_mnist_mpl.ddcdc8188b90.png)

Как видите, есть цифры с разными почерками. По мере того, как GAN изучает распределение данных, он также генерирует цифры с разными стилями рукописного ввода.

Теперь, когда вы подготовили обучающие данные, вы можете реализовать модели дискриминатора и генератора.

# Реализация Дискриминатора и Генератора
В этом случае дискриминатором является нейронная сеть MLP, которая принимает изображение 28 × 28 пикселей и обеспечивает вероятность того, что изображение принадлежит реальным обучающим данным.

Вы можете определить модель с помощью следующего кода:

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), 784)
        output = self.model(x)
        return output

Чтобы ввести коэффициенты изображения в нейронную сеть MLP, вы векторизуете их так, что нейронная сеть получает векторы с 784 коэффициентами.

Векторизация происходит в первой строке .forward (), так как вызов x.view () преобразует форму входного тензора. В этом случае исходная форма ввода x будет 32 × 1 × 28 × 28, где 32 - размер партии, который вы настроили. После преобразования форма x становится 32 × 784, причем каждая строка представляет коэффициенты изображения обучающего набора.

Чтобы запустить модель дискриминатора с использованием графического процессора, вы должны создать его экземпляр и отправить в графический процессор с помощью .to (). Чтобы использовать графический процессор, когда он доступен, вы можете отправить модель на объект устройства, который вы создали ранее:

In [ ]:
discriminator = Discriminator().to(device=device)

Поскольку генератор будет генерировать более сложные данные, необходимо увеличить размеры входных данных из скрытого пространства. В этом случае на генератор будет подан 100-мерный вход, и он будет обеспечивать выход с 784 коэффициентами, которые будут организованы в тензоре 28 × 28, представляющем изображение.

Вот полный код модели генератора:

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 784),
            nn.Tanh(),
        )

    def forward(self, x):
        output = self.model(x)
        output = output.view(x.size(0), 1, 28, 28)
        return output

generator = Generator().to(device=device)

В строке 12 вы используете гиперболическую касательную функцию Tanh () в качестве активации выходного слоя, поскольку выходные коэффициенты должны находиться в интервале от -1 до 1. В строке 20 вы создаете экземпляр генератора и отправляете его на устройство для использования. графический процессор, если таковой имеется.

Теперь, когда у вас есть определенные модели, вы будете обучать их, используя данные обучения.

# Обучение моделей
Для обучения моделей вам нужно определить параметры обучения и оптимизаторы, как вы делали в предыдущем примере:

In [ ]:
num_epochs = 50
loss_function = nn.BCELoss()

optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

Чтобы получить лучший результат, вы уменьшаете скорость обучения по сравнению с предыдущим примером. Вы также устанавливаете количество эпох на 50, чтобы сократить время обучения.

Цикл обучения очень похож на тот, который вы использовали в предыдущем примере. В выделенных строках вы отправляете данные тренировки на устройство для использования графического процессора, если доступно:

In [ ]:
for epoch in range(num_epochs):
    for n, (real_samples, mnist_labels) in enumerate(train_loader):
        # Data for training the discriminator
        real_samples = real_samples.to(device=device)
        real_samples_labels = torch.ones((batch_size, 1)).to(
            device=device
        )
        latent_space_samples = torch.randn((batch_size, 100)).to(
            device=device
        )
        generated_samples = generator(latent_space_samples)
        generated_samples_labels = torch.zeros((batch_size, 1)).to(
            device=device
        )
        all_samples = torch.cat((real_samples, generated_samples))
        all_samples_labels = torch.cat(
            (real_samples_labels, generated_samples_labels)
        )

        # Training the discriminator
        discriminator.zero_grad()
        output_discriminator = discriminator(all_samples)
        loss_discriminator = loss_function(
            output_discriminator, all_samples_labels
        )
        loss_discriminator.backward()
        optimizer_discriminator.step()

        # Data for training the generator
        latent_space_samples = torch.randn((batch_size, 100)).to(
            device=device
        )

        # Training the generator
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)
        output_discriminator_generated = discriminator(generated_samples)
        loss_generator = loss_function(
            output_discriminator_generated, real_samples_labels
        )
        loss_generator.backward()
        optimizer_generator.step()

        # Show loss
        if n == batch_size - 1:
            print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
            print(f"Epoch: {epoch} Loss G.: {loss_generator}")

Некоторые из тензоров не нужно явно отправлять в графический процессор с помощью устройства. Это относится к генерируемым_сэмплам в строке 11, которые уже будут отправлены в доступный графический процессор, поскольку latent_space_samples и генератор были отправлены в графический процессор ранее.

Поскольку в этом примере представлены более сложные модели, обучение может занять немного больше времени. После его завершения вы можете проверить результаты, сгенерировав несколько образцов рукописных цифр.

Проверка образцов, сгенерированных GAN
Чтобы сгенерировать рукописные цифры, вы должны взять несколько случайных выборок из скрытого пространства и передать их в генератор:

In [ ]:
latent_space_samples = torch.randn(batch_size, 100).to(device=device)
generated_samples = generator(latent_space_samples)

Чтобы построить сгенерированные_выборки, вам нужно переместить данные обратно в ЦП, если они работают на графическом процессоре. Для этого вы можете просто вызвать .cpu (). Как и раньше, вам также необходимо вызвать .detach () перед использованием Matplotlib для построения графика данных:

In [ ]:
generated_samples = generated_samples.cpu().detach()
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(generated_samples[i].reshape(28, 28), cmap="gray_r")
    plt.xticks([])
    plt.yticks([])

На выходе должны быть цифры, напоминающие тренировочные данные, как на следующем рисунке:

![](https://files.realpython.com/media/fig_generated_mnist_mpl.ecc483e9dd9d.png)

После пятидесяти эпох обучения есть несколько сгенерированных цифр, которые напоминают реальные. Вы можете улучшить результаты, рассматривая больше эпох обучения. Как и в предыдущем примере, используя фиксированный тензор выборок скрытого пространства и подавая его на генератор в конце каждой эпохи во время процесса обучения, вы можете визуализировать эволюцию обучения:

![](https://files.realpython.com/media/fig_gan_mnist.5d8784a85944.gif)

Вы можете видеть, что в начале тренировочного процесса сгенерированные изображения абсолютно случайны. По ходу обучения генератор изучает распределение реальных данных, и примерно через двадцать эпох некоторые сгенерированные цифры уже напоминают реальные данные.

# Заключение
Поздравляем! Вы узнали, как реализовать свои собственные генеративные состязательные сети. Сначала вы прошли игрушечный пример, чтобы понять структуру GAN, прежде чем погрузиться в практическое приложение, которое генерирует изображения рукописных цифр.

Вы видели, что, несмотря на сложность GAN, интегрированные среды машинного обучения, такие как PyTorch, делают реализацию более простой, предлагая автоматическую дифференциацию и простую настройку графического процессора.

# Дальнейшее чтение